In [1]:
import pandas as pd
import numpy as np


In [2]:
def read_in_logfile(path, vid_lengths):
    pd.read_csv(path, sep='\t', skip_rows=3)


def get_blocks(log,vid_info):
    #identifies the video trial types (as opposed to button press events etc)
    mask = ["vid" in log['Code'][i] for i in range(0,log.shape[0])]
    #this isnt totally right lol
    #creates the dataframe with onset times and event types
    df = pd.DataFrame({'onset':log.loc[mask]['Time'], 
                  'trial_type':log.loc[mask]['Event Type'], 
                  'movie_name':log.loc[mask]['Code']})
    
    #adds trial type info
    df['trial_type']=df['movie_name'].apply(lambda x: "circle_block" if "cvid" in x else "EA_block")

    #add durations and convert them into the units here? ms??
    df['duration']=df['movie_name'].apply(lambda x: int(vid_info[x]['duration'])*10000 if x in vid_info else "n/a")

    #I don't actually know what the stim files are called for the circle ones - also these names aren't exact,gotta figure out a way to get exact file names
    df['stim_file']=df['movie_name'].apply(lambda x: vid_info[x]['stim_file'] if x in vid_info else "n/a") 
    
    
    df['end']=df['onset'].apply(lambda x: x + df['duration'] "cvid" in x else "EA_block")

        
    return(df)


    
def format_vid_info(vid):
    vid.columns = map(str.lower, vid.columns)
    vid = vid.rename(index={0:"stim_file", 1:"duration"})
    vid = vid.to_dict()
    return(vid)



def get_ratings(log):
    #the times in this row are EXTREMELY close to the other times. This isn't EEG, I think we're prolly ok
    rating_mask = ["rating" in log['Code'][i] for i in range(0,log.shape[0])]  
    #RT_mask=  ["Response" in log['Event Type'][i] and log['Code'][i]!="101"  for i in range(0,log.shape[0]-1)]  #this is from when i was doing it the response time way, but idk how i feel abt that

    #so now this grabs the timestamp from the row before (which is the actual onset) then applies the rating mask to that list of values
    df = pd.DataFrame({'onset':log['Time'].shift(1).loc[rating_mask].values, 'participant_rating':log.loc[rating_mask]['Code'].values, 'event_type':'button_press', 'duration':0})    
    
    #TODO: probably remove this from this function and rewrite it in the place where i combine the ratings and block info
    #df['rating_duration'] = df.onset.shift(-1)-df.onset #this isnt totally correct bc of the stuff.

    return(df)



In [31]:
#Reads in the log, skipping the first three preamble lines
log=pd.read_csv('/projects/gherman/Experimenting_notebooks/SPN01_CMH_0001-UCLAEmpAcc_part1.log', sep='\t', skiprows=3)


vid_in = pd.read_csv('EA-vid-lengths.csv')

vid_info = format_vid_info(vid_in)
blocks = get_blocks(log, vid_info)
ratings = get_ratings(log)

#add the ratings and the block values together, then sort them and make the index numbers sequential
combo=blocks.append(ratings).sort_values("onset").reset_index(drop=True)
combo




#find what index each block starts and ends at, then does some stupid formatting stuff to flatten the array
t=np.array(np.where(pd.notnull(combo['trial_type']))).ravel()

#adds the end of the last trial so i can get starts and ends of everything #delete this if im doing it this way
#t = np.append(t,len(combo["trial_type"])-1)

#creates a dict with trials and start/ends. Next it needs to differentiate between circles and EA and then get the nice values.
[{(combo['trial_type'][t[i]],combo['stim_file'][t[i]]): {'start':t[i],'end':t[i+1]-1}} for i in range(len(t)-1)]
#^okay, so maybe instead it makes sense to include the actual times... or maybe to create an actual df with info about all this.

#df = pd.DataFrame({'tria':log.loc[mask]['Time'], 
#        'trial_type':log.loc[mask]['Event Type'], 
#        'movie_name':log.loc[mask]['Code']})

#df1 = df[['a','b']] for selecting columns


t=np.array(np.where(pd.notnull(combo['trial_type']))).ravel()

    
combo.loc[t]['trial_type']
combo.loc[t]['stim_file']
combo.loc[t]['onset']
#combo.loc[t]['onset']+combo.loc[t]['duration']

 #adds trial type info
  #  df['trial_type']=df['movie_name'].apply(lambda x: "circle_block" if "cvid" in x else "EA_block")

combo

duration    event_type movie_name      onset participant_rating  \
0    1700000           NaN      vid_4  1181706.0                NaN   
1          0  button_press        NaN  1307123.0   rating_equal_to6   
2          0  button_press        NaN  1356642.0   rating_equal_to5   
3          0  button_press        NaN  1369317.0   rating_equal_to4   
4          0  button_press        NaN  1388159.0   rating_equal_to3   
5          0  button_press        NaN  1446766.0   rating_equal_to4   
6          0  button_press        NaN  1472211.0   rating_equal_to5   
7          0  button_press        NaN  1532008.0   rating_equal_to6   
8          0  button_press        NaN  1596274.0   rating_equal_to7   
9          0  button_press        NaN  1615081.0   rating_equal_to8   
10         0  button_press        NaN  1644045.0   rating_equal_to9   
11         0  button_press        NaN  2660894.0   rating_equal_to8   
12         0  button_press        NaN  2825279.0   rating_equal_to9   
13    400000           NaN     cvid_1  2938971.0                NaN   
14         0  button_press        NaN  2966329.0   rating_equal_to4   
15         0  button_press        NaN  2983848.0   rating_equal_to3   
16         0  button_press        NaN  3028985.0   rating_equal_to4   
17         0  button_press        NaN  3031479.0   rating_equal_to5   
18         0  button_press        NaN  3033719.0   rating_equal_to6   
19         0  button_press        NaN  3067962.0   rating_equal_to5   
20         0  button_press        NaN  3070540.0   rating_equal_to4   
21         0  button_press        NaN  3072853.0   rating_equal_to3   
22         0  button_press        NaN  3091247.0   rating_equal_to2   
23         0  button_press        NaN  3100331.0   rating_equal_to3   
24         0  button_press        NaN  3124660.0   rating_equal_to4   
25         0  button_press        NaN  3150246.0   rating_equal_to5   
26         0  button_press        NaN  3152754.0   rating_equal_to6   
27         0  button_press        NaN  3154662.0   rating_equal_to7   
28         0  button_press        NaN  3156450.0   rating_equal_to8   
29         0  button_press        NaN  3197681.0   rating_equal_to9   
..       ...           ...        ...        ...                ...   
68         0  button_press        NaN  5045527.0   rating_equal_to3   
69         0  button_press        NaN  5067615.0   rating_equal_to2   
70         0  button_press        NaN  5070130.0   rating_equal_to1   
71         0  button_press        NaN  5100981.0   rating_equal_to2   
72         0  button_press        NaN  5147155.0   rating_equal_to3   
73         0  button_press        NaN  5149683.0   rating_equal_to4   
74         0  button_press        NaN  5159331.0   rating_equal_to5   
75         0  button_press        NaN  5173121.0   rating_equal_to6   
76         0  button_press        NaN  5184239.0   rating_equal_to5   
77         0  button_press        NaN  5189275.0   rating_equal_to4   
78         0  button_press        NaN  5194508.0   rating_equal_to3   
79         0  button_press        NaN  5196812.0   rating_equal_to2   
80         0  button_press        NaN  5199251.0   rating_equal_to1   
81         0  button_press        NaN  5212084.0   rating_equal_to2   
82         0  button_press        NaN  5230332.0   rating_equal_to1   
83         0  button_press        NaN  5263838.0   rating_equal_to2   
84         0  button_press        NaN  5272876.0   rating_equal_to3   
85         0  button_press        NaN  5353836.0   rating_equal_to4   
86   1110000           NaN      vid_5  5353836.0                NaN   
87         0  button_press        NaN  5412504.0   rating_equal_to6   
88         0  button_press        NaN  5428428.0   rating_equal_to7   
89         0  button_press        NaN  5448886.0   rating_equal_to8   
90         0  button_press        NaN  5478000.0   rating_equal_to9   
91         0  button_press        NaN  6297184.0   rating_equal_to8   
92         0  button_press  

In [48]:
[{(combo['trial_type'][t[i]],combo['stim_file'][t[i]]): {'start':(t[i], combo['onset'][t[i]]),'end':(t[i+1]-1,combo['onset'][t[i]]+combo['duration'][t[i]])}} for i in range(len(t)-1)]
combo


mask = pd.notnull(combo['trial_type'])
combo[['onset', 'duration']].sum(axis=1).where(mask==True,0)

0     2881706.0
1           0.0
2           0.0
3           0.0
4           0.0
5           0.0
6           0.0
7           0.0
8           0.0
9           0.0
10          0.0
11          0.0
12          0.0
13    3338971.0
14          0.0
15          0.0
16          0.0
17          0.0
18          0.0
19          0.0
20          0.0
21          0.0
22          0.0
23          0.0
24          0.0
25          0.0
26          0.0
27          0.0
28          0.0
29          0.0
        ...    
68          0.0
69          0.0
70          0.0
71          0.0
72          0.0
73          0.0
74          0.0
75          0.0
76          0.0
77          0.0
78          0.0
79          0.0
80          0.0
81          0.0
82          0.0
83          0.0
84          0.0
85          0.0
86    6463836.0
87          0.0
88          0.0
89          0.0
90          0.0
91          0.0
92          0.0
93          0.0
94          0.0
95          0.0
96          0.0
97          0.0
dtype: float64

In [19]:

#Time col contains the button press onset time in event_type=response, Code col 
#contains the button press' value in rating_mask

#I am not entirely sure that I should be getting the time from the row before, but it kind of makes sense to take the most straightforward one

#101 is the event code for some initial button press or response. We can ignore it. (AT LEAST acc. to the CAMH
#scan. Please cross-reference with others before finalizing)


#the times in this row are EXTREMELY close to the other times. This isn't EEG, I think we're prolly ok
rating_mask = ["rating" in log['Code'][i] for i in range(0,log.shape[0])]  
#RT_mask=  ["Response" in log['Event Type'][i] and log['Code'][i]!="101"  for i in range(0,log.shape[0]-1)]  #this is from when i was doing it the response time way, but idk how i feel abt that

#so now this grabs the timestamp from the row before (which is the actual onset) then applies the rating mask to that list of values
df = pd.DataFrame({'onset':log['Time'].shift(1).loc[rating_mask].values, 'participant_rating':log.loc[rating_mask]['Code'].values, 'event_type':'button_press', 'duration':0})    

df['rating_duration'] = df.onset.shift(-1)-df.onset #this isnt totally correct bc of the stuff.



In [57]:
df2=blocks.append(df)

df2.sort_values("onset")

log

rating_mask = ["rating" in log['Code'][i] for i in range(0,log.shape[0])]  
log['Time'].shift(1).loc[rating_mask].values

array([ 1307123.,  1356642.,  1369317.,  1388159.,  1446766.,  1472211.,
        1532008.,  1596274.,  1615081.,  1644045.,  2660894.,  2825279.,
        2966329.,  2983848.,  3028985.,  3031479.,  3033719.,  3067962.,
        3070540.,  3072853.,  3091247.,  3100331.,  3124660.,  3150246.,
        3152754.,  3154662.,  3156450.,  3197681.,  3231248.,  3234079.,
        3236202.,  3238219.,  3240178.,  3242267.,  3263548.,  3265576.,
        3267439.,  3269304.,  3278456.,  3305389.,  3390657.,  3479701.,
        3482813.,  3787292.,  3810557.,  3931991.,  3941271.,  3949061.,
        3964947.,  4464758.,  4500306.,  4502755.,  4512139.,  4514283.,
        4816652.,  4819259.,  4902353.,  4911113.,  4919457.,  4928720.,
        4945514.,  4982039.,  4984537.,  5024817.,  5045527.,  5067615.,
        5070130.,  5100981.,  5147155.,  5149683.,  5159331.,  5173121.,
        5184239.,  5189275.,  5194508.,  5196812.,  5199251.,  5212084.,
        5230332.,  5263838.,  5272876.,  5353836., 

In [56]:
rating_mask = ["rating" in log['Code'][i] for i in range(0,log.shape[0])]  
log['Time'].shift(1).loc[rating_mask]


5      1307123.0
7      1356642.0
9      1369317.0
11     1388159.0
13     1446766.0
15     1472211.0
17     1532008.0
19     1596274.0
21     1615081.0
23     1644045.0
25     2660894.0
27     2825279.0
31     2966329.0
33     2983848.0
35     3028985.0
37     3031479.0
39     3033719.0
41     3067962.0
43     3070540.0
45     3072853.0
47     3091247.0
49     3100331.0
51     3124660.0
53     3150246.0
55     3152754.0
57     3154662.0
59     3156450.0
61     3197681.0
63     3231248.0
65     3234079.0
         ...    
133    5024817.0
135    5045527.0
137    5067615.0
139    5070130.0
141    5100981.0
143    5147155.0
145    5149683.0
147    5159331.0
149    5173121.0
151    5184239.0
153    5189275.0
155    5194508.0
157    5196812.0
159    5199251.0
161    5212084.0
163    5230332.0
165    5263838.0
167    5272876.0
169    5353836.0
171    5412504.0
173    5428428.0
175    5448886.0
177    5478000.0
179    6297184.0
181    6300223.0
183    6318245.0
185    6320923.0
187    6337374

In [ ]:
#HERE LIES THE CHUNK OF JUNK IM MOVING TO THE END

#(combo['trial_type'][t[0]],combo['trial_type'][t[1]-1])
#pd.notnull(combo['trial_type'])
#(starts,ends)=[(t[i],t[i+1]-1) for i in range(len(t)-1)]

#combo.sort_values("onset")
#uses isna in some versions
#combo.isnull()

#combo.sort_values("onset")


#log['Time']=log['Time'].astype(str) #can i read it in initially with strings only? like stringsAsFactors=T in R?

#Finds the start and end times of blocks


#okay so Time from row n-1 + TTime from row n = Time in row n !!! 
#We will subtract Duration of MRI start from all times to give us the time
#(block_starts, block_types) = (log['Time'][] 
                               
 #                              df.loc[df['B'], 'A']

#f['c2'] = df['c1'].apply(lambda x: 10 if x == 'Value' else x)

#def get_button_press(log):

#Time col contains the button press onset time in event_type=response, Code col 
#contains the button press' value in rating_mask

#101 is the event code for some initial button press or response. We can ignore it. (AT LEAST acc. to the CAMH
#scan. Please cross-reference with others before finalizing)

#log['Time'] = log['Time'].astype(str)
#rating_mask = ["rating" in log['Time'][i] for i in range(0,log.shape[0]-1)]  
#response_mask=  ["rating" in log['Code'][i] for i in range(0,log.shape[0]-1)]  

#or "Response" in log['Code'][i]
    
#df = pd.DataFrame({'onset':log.loc[rating_mask]['Time'], 'rating':log.loc[response_mask]['Code']})    

#df
    
#pd.DataFrame{'col1': [1, 2], 
           #  'col2': [3, 4]}

#[[log['Code'][i], log['Time'][i], log['Event Type'][i]] for i in range(0,log.shape[0]-1) if("vid" in log['Code'][i])]
